In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=eb9ce5a2be02dcc09fd4502cb59f87001ae47535a40fa1ea067a150cab56c0bd
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os
import time

# Initialize Spark Session
spark = SparkSession.builder.master("local").appName("LogMonitoring").getOrCreate()

# Directory to simulate streaming log data
log_dir = "/content/log_data" # Changed to a directory
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

# Read streaming data from the log directory
log_data = spark.readStream.text(log_dir)

# Filter for ERROR and WARNING messages
filtered_logs = log_data.filter((col("value").contains("ERROR")) | (col("value").contains("WARNING")))

# Output the filtered logs to the console
query = filtered_logs.writeStream.outputMode("append").format("console").start()

# Simulate log file creation
for i in range(5):
    with open(f"{log_dir}/log_file_{i}.txt", "w") as f: # Write files into the directory
        f.write(f"This is log entry {i}\nWARNING: Something went wrong!\n")
    time.sleep(10)

# Wait for the streaming to finish
query.awaitTermination(60)

False